In [25]:
import os
import shutil
import glob
import numpy as np 
import pandas as pd
import pretty_midi
import pypianoroll
# import tables
from music21 import converter, instrument, note, chord, stream
import music21
import librosa
import librosa.display
import matplotlib.pyplot as plt
from keras.utils import np_utils
import json
import IPython.display
from datetime import datetime
import random

import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from tqdm.notebook import tqdm, trange  

import random
import itertools
root_dir = r'C:\Users\ahdz3\Desktop\jazzmidis'
data_dir = root_dir + '\Jazz Midi'
# music_dataset_lpd_dir = root_dir + '/Music Dataset/midis/lmd_matched'

In [26]:
print(root_dir)
midi_info = pd.read_csv(root_dir + '\Jazz-midi.csv')
# midi_info.head()

C:\Users\ahdz3\Desktop\jazzmidis


In [ ]:
# view which tracks are in our jazz midi files
j = 0

for msd_file_name in midi_info['Name']:


  # create path to midi file
  midi_path = os.path.join(data_dir, msd_file_name)
  
  # continue if file is corrupted
  try:
    multitrack = pypianoroll.read(midi_path)
  except:
    print(msd_file_name, ' is corrupted')
    continue

  multitrack.set_resolution(2).pad_to_same()

  print(j, ': ', multitrack.tracks, '\n\n')
  j += 1


In [ ]:
combined_pianorolls = []
i = 0

# change root_dir to training set afterwards
for msd_file_name in midi_info['Name']:

   # create path to midi file
  midi_path = os.path.join(data_dir, msd_file_name)
  
  # continue if file is corrupted
  try:
    multitrack = pypianoroll.read(midi_path)
  except:
    print(msd_file_name, ' is corrupted')
    continue

  multitrack.set_resolution(2).pad_to_same()

  # Piano, Guitar, Bass, Strings, Drums
  # Splitting into different parts

  # RESOLVE: original code uses LPD-5 standardized pianorolls that have all 5 parts

  parts = {'piano_part': None, 'guitar_part': None, 'bass_part': None, 'strings_part': None, 'drums_part': None}
  song_length = None
  empty_array = None
  has_empty_parts = False
  for track in multitrack.tracks:
    if 'Drums' in track.name:
      parts['drums_part'] = track.pianoroll
    if 'Piano' in track.name:
      parts['piano_part'] = track.pianoroll
    if 'Guitar' in track.name:
      parts['guitar_part'] = track.pianoroll
    if 'Bass' in track.name:
      parts['bass_part'] = track.pianoroll
    if 'Strings' in track.name:
      parts['strings_part'] = track.pianoroll
    if track.pianoroll.shape[0] > 0:
      empty_array = np.zeros_like(track.pianoroll)


  for k,v in parts.items():
    if (v is None) or (v.shape[0] == 0):
      parts[k] = empty_array.copy()
      has_empty_parts = True

  # Stack all together - Piano, Guitar, Bass, Strings, Drums
  combined_pianoroll = torch.tensor([parts['piano_part'], parts['guitar_part'], parts['bass_part'], parts['strings_part'], parts['drums_part']])

  # These contain velocity information - the force with which the notes are hit - which can be standardized to 0/1 if we want (to compress)
  if has_empty_parts == False:
    combined_pianorolls.append(combined_pianoroll)
    i+=1
    print(i)

  if i == 1000:
    break

In [ ]:
torch.save(combined_pianorolls, os.path.join(root_dir, 'jazz_combined_pianorolls', 'jazz_rolls_vae.pt'))
pianoroll_lengths = torch.tensor(pianoroll_lengths)
torch.save(pianoroll_lengths, os.path.join(root_dir, 'jazz_combined_pianorolls', 'jazz_rolls_vae_lengths.pt'))